In [70]:
from helpers import (
    preprocessing
)
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, IntegerType, DateType, FloatType, ArrayType, LongType, MapType
import warnings
import numpy as np 
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
warnings.filterwarnings('ignore')

In [80]:
!pip install bert-for-tf2
!pip install -U transformers

In [71]:
# base de dados do twitter ja classificada com sentimentos
path = "/home/daholive/Documents/twitter_ellection_brazil_v2/datasource/raw_kaggle/NoThemeTweets.csv"

In [72]:
# dataframe twitter com sentimentos classificados
df = pd.read_csv(path, error_bad_lines=False, sep=',')

In [73]:
def get_treated_data(
    dataset, 
    cols, 
    cols_drop = [], 
    col_to_change='sentiment', 
    val_col_change = {"Negativo": 0, "Positivo":1}
):
 
    # 2. Rename columns
    dataset.columns = cols
    
    # 3. Drop columns not needed
    dataset.drop(cols_drop, axis=1, inplace=True)
    
    # 3.1 Drop all rows with at least one element is missing
    dataset.dropna()
    
    # 4. Convert setiments from "Negative/Positive" to "0/1" 
    dataset.replace({col_to_change: val_col_change}, inplace=True)
    
    # Return our dataset
    return dataset

In [74]:
default_cols = ["id", "text", "date", "sentiment", "query"]

default_drop_cols = ["id", "date", "query"]

# class_names = ['Negativo', 'Positive']
df = get_treated_data(df, default_cols, cols_drop = default_drop_cols)

In [75]:
# Take a peek at the dataset
df["sentiment"].value_counts(normalize=True)

0    0.665179
1    0.334821
Name: sentiment, dtype: float64

In [76]:
def preprocess_text(text):
    
    # Not needed to be imported globally
    from bs4 import BeautifulSoup
    import re
    text = BeautifulSoup(text, "lxml").get_text()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) # Remove urls
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-Z.!?']", ' ', text)
    text = re.sub(r" +", ' ', text)
    
    return text

In [77]:
df['text'] = df['text'].apply(lambda text: preprocess_text(text))

In [32]:
df.head()

,text,sentiment
0,para eu ir,Positivo
1,O meu like eu j dei na poca,Positivo
2,Eu s queria conseguir comer alguma coisa pra p...,Positivo
3,D que lindo dia !,Positivo
4,Resmungao Pq da pr jeito!! uma oferta ha q ap...,Positivo


In [33]:
print("The number of rows and columns in the dataset is: {}".format(df.shape))

The number of rows and columns in the dataset is: (785814, 2)


In [34]:
# Identify missing values
df.apply(lambda x: sum(x.isnull()), axis=0)

text         0
sentiment    0
dtype: int64

In [35]:
# Check the target class balance
df["sentiment"].value_counts(normalize=True)

Negativo    0.665179
Positivo    0.334821
Name: sentiment, dtype: float64

In [89]:
!conda install -y PyTorch

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::tensorflow-gpu==2.4.1=h30adc30_0
  - defaults/linux-64::tensorflow==2.4.1=gpu_py38h8a7d6ce_0
  - defaults/linux-64::tensorflow-base==2.4.1=gpu_py38h29c2da4_0
  - defaults/linux-64::h5py==2.10.0=py38hd6299e0_1
  - defaults/linux-64::numpy==1.21.2=py38h20f2e39_0
  - defaults/linux-64::mkl_random==1.2.2=py38h51133e4_0
  - defaults/noarch::tensorflow-estimator==2.6.0=pyh7b7c402_0
  - defaults/noarch::opt_einsum==3.3.0=pyhd3eb1b0_1
  - defaults/noarch::tensorboard==2.4.0=pyhc547734_0
  - defaults/noarch::keras-preprocessing==1.1.2=pyhd3eb1b0_0
  - defaults/linux-64::mkl_fft==1.3.1=py38hd3c417c_0
done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment loca

In [92]:
from transformers import AutoModel, AutoTokenizer

In [93]:
# BERT Base
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModel".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [94]:
import torch
torch.cuda.is_available()

False